# CIFAR-10

In [1]:
# fast ai installations
!pip install fastai==0.7.0
!pip install torchtext==0.2.3

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 496.4MB 35kB/s 
    100% |████████████████████████████████| 71kB 22.8MB/s 
    100% |████████████████████████████████| 1.5MB 2.5MB/s 
    100% |████████████████████████████████| 184kB 26.4MB/s 
    100% |████████████████████████████████| 3.6MB 1.2MB/s 
    100% |████████████████████████████████| 11.6MB 1.5MB/s 
    100% |████████████████████████████████| 92kB 24.8MB/s 
    100% |████████████████████████████████| 61kB 22.1MB/s 
    100% |████████████████████████████████| 81kB 11.0MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Running setu

In [0]:
from fastai.conv_learner import *

In [4]:
# train data upload
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.7z to train.7z
User uploaded file "train.7z" with length 109723070 bytes


In [13]:
# trainLabels.csv upload
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving trainLabels.csv to trainLabels.csv
User uploaded file "trainLabels.csv" with length 588903 bytes


In [5]:
!ls

cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb	 PyTorch-YOLOv3
cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb.1  sample_data
darknet							 train.7z


In [8]:
!apt-get install p7zip-full
!p7zip -d train.7z

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 109723070 bytes (105 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 626 - train/10561.png                            2% 1079 - train/1097.png                            2% 1402 - train/1126.png                          

In [12]:
!ls train | head

10000.png
10001.png
10002.png
10003.png
10004.png
10005.png
10006.png
10007.png
10008.png
10009.png


In [14]:
!ls

cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb	 sample_data
cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb.1  train
darknet							 trainLabels.csv
PyTorch-YOLOv3


In [17]:
%cd  ..
!ls

/
bin	    content  etc   lib32  mnt	root  srv    tmp    var
boot	    datalab  home  lib64  opt	run   swift  tools
colabtools  dev      lib   media  proc	sbin  sys    usr


In [0]:
PATH = Path("content/")
os.makedirs(PATH, exist_ok=True)

In [47]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

0.3.1
True
True


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = np.array([ 0.4914, 0.48216, 0.44653]), np.array([0.24703, 0.24349, 0.26159])

In [0]:
num_workers = num_cpus() // 2
bs=256
sz=32

In [50]:
label_csv = f'{PATH}/trainLabels.csv'
label_csv

'content/trainLabels.csv'

In [51]:
n = len(list(open(label_csv)))-1
n

50000

In [52]:
val_idxs = get_cv_idxs(n)
val_idxs

array([33553,  9427,   199, ..., 18707, 15200,  5857])

In [0]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)

In [0]:
# data = ImageClassifierData.from_paths(PATH, trn_name='train/', val_name='test', tfms=tfms, bs=bs)
data = ImageClassifierData.from_csv(PATH, 'train/', label_csv, tfms=tfms, suffix='.png', val_idxs=val_idxs)

In [0]:
def conv_layer(ni, nf, ks=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=ks, bias=False, stride=stride, padding=ks//2),
        nn.BatchNorm2d(nf, momentum=0.01),
        nn.LeakyReLU(negative_slope=0.1, inplace=True))

In [0]:
class ResLayer(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.conv1=conv_layer(ni, ni//2, ks=1)
        self.conv2=conv_layer(ni//2, ni, ks=3)
        
    def forward(self, x): return self.conv2(self.conv1(x)) + x

In [0]:
class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [conv_layer(ch_in, ch_in*2, stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]
    
    def __init__(self, num_blocks, num_classes, nf=32):
        super().__init__()
        layers = [conv_layer(3, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            layers += self.make_group_layer(nf, nb, stride=2-(i==1))
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, num_classes)]
        self.layers = nn.Sequential(*layers)
        
    def forward(self,x): return self.layers(x)

In [0]:
m = Darknet([1, 2, 4, 6, 3], num_classes=10, nf=32)
m = nn.DataParallel(m, device_ids=None)
# if you have several GPUs for true parallel processing enable
# m = nn.DataParallel(m, device_ids=[1, 2, 3])

In [0]:
lr = 1.3

In [0]:
learn = ConvLearner.from_model_data(m, data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
wd=1e-4

In [73]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

epoch      trn_loss   val_loss   accuracy   
    0      2.091498   2.100721   0.1985    
    1      1.842176   1.932684   0.274     
    2      1.667993   2.449561   0.1845    
    3      1.46315    2.357106   0.3116    
    4      1.365894   1.363215   0.516     
    5      1.286649   1.616203   0.5122    
    6      1.221727   1.519177   0.4983    
    7      1.153103   1.170713   0.6162    
    8      1.075718   1.919413   0.4529    
    9      0.995893   2.356375   0.3817    
    10     1.030227   2.64268    0.3779    
    11     1.0038     1.547633   0.5089    
    12     0.981379   1.642641   0.5178    
    13     0.9332     1.168541   0.604     
    14     0.910082   1.960581   0.4887    
    15     0.903902   1.385649   0.5392    
    16     0.881015   1.113259   0.6243    
    17     0.838519   1.033334   0.6581    
    18     0.827389   1.382058   0.5609    
    19     0.793658   1.141158   0.6261    
    20     0.720368   0.901192   0.6844    
    21     0.664654   1.104266 

[array([0.31962]), 0.8923]

In [74]:
# DP: m = WideResNet(depth=22, num_classes=10, widen_factor=6, dropRate=0.)
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

epoch      trn_loss   val_loss   accuracy   
    0      0.260877   0.339372   0.8851    



[array([0.33937]), 0.8851]

In [0]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20, 20, 0.95, 0.85))

In [0]:
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

In [0]:
%time learn.fit(lr, 1, wds=wd, cycle_len=40, use_clr_beta=(10, 15, 0.95, 0.85))

In [0]:
learn.fit(lr/10, 1, wds=wd, cycle_len=1, use_clr_beta=(100, 1, 0.9, 0.8))

In [0]:
%time learn.fit(1., 1, wds=wd, cycle_len=30, use_clr_beta=(10, 25, 0.95, 0.85))

In [0]:
%time learn.fit(lr, 1, wds=wd, cycle_len=40, use_clr_beta=(100, 15, 0.95, 0.85))

In [0]:
# darknet 2222 lr 1.3 65 cl
%time learn.fit(lr, 1, wds=wd, cycle_len=65, use_clr_beta=(30, 20, 0.95, 0.85))